# Wellbore Paths in PyVista 

This notebook was adapted from a resource found online, but apparently now removed.

This is an attempt to use pyvista and/or pvgeo to visualise borehole log data in the browser. 


Now we are going to import `pandas` for reading in our well data, `numpy` for making stratigraphic surfaces, `pyvista` and `PVGeo` for visualization. Also we are going to set the `panel.extension` to `'vtk'` and set the `pyvista` theme to `document

In [ ]:
import pandas as pd
import numpy as np
import pyvista as pv
import panel
import PVGeo

panel.extension("vtk")
pv.set_plot_theme("document")
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import sys
import os

ela_from_source = True

if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\Users\SUD011\Documents\pyela-sudhir'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/github_jm/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *

In [ ]:
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'


## Global settings and preprocessed data


In [ ]:
if ('ELA_DATA' in os.environ):
    data_path = os.environ['ELA_DATA']
elif sys.platform == 'win32':
    data_path = r'C:\data\Lithology'
else:
    username = os.environ['USER']
    data_path = os.path.join('/home', username, 'data', 'Lithology')

data_in_dir = os.path.join(data_path, 'data_in')
shapes_in_dir = os.path.join(data_in_dir, 'Shapes')

df = pd.read_pickle(os.path.join(data_path, 'data_inter', 'preprocessed.pkl'))
# df = pd.read_pickle(os.path.join(data_path, 'data_inter', 'above_leederville.pkl'))

In [ ]:
len(df)

Let's read in the data using `pandas` and get a quick look at it

In [ ]:
data = df
data.head()

Let's get the names of the wells from the data and assign them to a variable

In [ ]:
scaled_heights_colname = 'scaled_z'
data[scaled_heights_colname]  = data[DEPTH_FROM_AHD_COL] * 50

In [ ]:
site_ref_colname='name'
#data.index.values.astype(str)
data[site_ref_colname]=data.index.values.astype(str)

In [ ]:
data.head()

In [ ]:
wells = data.name.unique()

Now we want to go through the data and build a `dict` for each well

In [ ]:
# split out the wells into a dictionary
well_dict = {}
for well in wells:
    well_dict["{0}".format(well)] = data[data.name == well]

Next let's take each well in the `well_dict` and get the points referenced to one another so they all plot up in real space relative to one another. We are then creating a `numpy` array with the `'easting'` (x-value), the `'northing'` (y-value), and the `depth` which is our true vertical depth. We now have each entry in the `points_dict` with an x, y, and z value.

In [ ]:



# now turn the wells into points
points_dict = {}
for points in well_dict:
    points_dict["{0}".format(points)] = np.array(
        list(
            zip(
                well_dict[points].Easting ,
                well_dict[points].Northing,
                well_dict[points][scaled_heights_colname],
            )
        )
    )

In [ ]:
well_dict[points]

From the points we want to create `PolyData` to plot in `PyVista`. To create this `PolyData` we use `PVGeo.points_to_poly_data()` on each entry in the `points_dict`. Then we use `PVGeo.filters.AddCellConnToPoints` to create cells connecting the points. At this point we have everything we need to plot up wellbore trajectories. But let's have some fun!

In [ ]:
# now turn points into polydata
lines_dict = {}
for lines in points_dict:
    poly = PVGeo.points_to_poly_data(points_dict[lines])
    lines_dict[
        "{0}".format(lines)
    ] = PVGeo.filters.AddCellConnToPoints().apply(poly)

In [ ]:
lines_dict['61470318']

To color and size the `PolyData` by a value we use the `GR` column from the original dataset. We assigne the `GR` gamma-ray values to each `PolyData` object meaning we now have `DataArrays` for each well that we can use to color our wellbore. Finally, we use `.tube()` to create tubes at each point with radius 10 that scale with the `GR` gamma-ray values. 

In [ ]:
litho_class_col = 'Lithology_1_num'

In [ ]:
path = '61470318'
data[data[site_ref_colname] == path][litho_class_col].values

In [ ]:
path = '61400032'
data[data[site_ref_colname] == path][litho_class_col].values

In [ ]:
lines_dict_tmp = {}

for path in lines_dict:
    vals = data[data[site_ref_colname] == path][litho_class_col].values
    lines_dict[path]["GR"] = vals
    lines_dict[path].tube(radius=10, scalars="GR", inplace=True)
    if len(vals) > 1:
        lines_dict_tmp[path] = lines_dict[path]
        

In [ ]:
lines_dict = lines_dict_tmp

In [ ]:
len(lines_dict)

In [ ]:
path = '61470318'
lines_dict[path]

In [ ]:
path = '61400032'
lines_dict[path]
len(data[data[site_ref_colname] == path][litho_class_col].values)

We could plot up the wellbores at this point and have a great plot, but let's make some stratigraphic surfaces. I grabbed some tops from the [COGCC](https://cogcc.state.co.us) website for one of these wells. To make surfaces we need `x` and `y` values spanning the area we are interested in. So I use the minimum and maximum from the dataset and then add 1,000 feet in the x and y direction for the surfaces. Next we use `meshgrid` and create our grid. Then for each surface I create an empty array using `np.empty()` and `.fill` it with the depth to each top. For the last step for each surface we use `StructuredGrid` to create a structured `PolyData` grid. Here we have the Niobrara, Shannon, Sussex, Parkman, and Pierre formation tops.

In [ ]:
data.Easting.max() - data.Easting.min(),  (data.Northing.max() - data.Northing.min())

In [ ]:
x = np.arange(
    data.Easting.min() - 1000,
    data.Easting.max() + 1000,
    100,
)
y = np.arange(
    data.Northing.min() - 1000,
    data.Northing.max() + 1000,
    100,
)

In [ ]:
x.shape, y.shape

In [ ]:
x, y = np.meshgrid(x, y)
r = np.sqrt(x ** 2 + y ** 2)

In [ ]:
nio = np.empty(r.shape)
nio.fill(-5930)
niobraragrid = pv.StructuredGrid(x, y, nio)

shn = np.empty(r.shape)
shn.fill(-4454)
shannongrid = pv.StructuredGrid(x, y, shn)

ssx = np.empty(r.shape)
ssx.fill(-4009)
sussexgrid = pv.StructuredGrid(x, y, ssx)

prk = np.empty(r.shape)
prk.fill(-3332)
parkmangrid = pv.StructuredGrid(x, y, prk)

pie = np.empty(r.shape)
pie.fill(-2267)
pierregrid = pv.StructuredGrid(x, y, pie)

All that work and now we get so see how it turned out! We start by creating our `Plotter` and then `add_mesh` for each pice of `PolyData` that we have created so far. Lastly, we call `.show()` and we have an interactive set of wells!

In [ ]:
plotter = pv.Plotter()
for well in lines_dict:
    plotter.add_mesh(lines_dict[well])

    
#plotter.add_mesh(niobraragrid, opacity=0.5, color="white")
#plotter.add_mesh(shannongrid, opacity=0.5, color="black")
#plotter.add_mesh(sussexgrid, opacity=0.5, color="purple")
#plotter.add_mesh(parkmangrid, opacity=0.5, color="orange")
#plotter.add_mesh(pierregrid, opacity=0.5, color="green")

plotter.show()

In [ ]:
well, lines_dict[well]

In [ ]:
# sphinx_gallery_thumbnail_number = 2
import pyvista as pv
from pyvista import examples# sphinx_gallery_thumbnail_number = 2

In [ ]:
type(examples)

In [ ]:
# Load St Helens DEM and warp the topography
mesh = examples.download_st_helens().warp_by_scalar()

# First a default plot with jet colormap
p = pv.Plotter()
# Add the data, use active scalar for coloring, and show the scalar bar
p.add_mesh(mesh)
# Display the scene
p.show()

In [ ]:
mesh

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(pv.Sphere(), name='mydata')
#plotter.add_mesh(pv.Plane(), name='mydata')
# Only the Plane is shown!
plotter.show()

This notebook is licensed as CC-BY, use and share to your hearts content.

In [ ]:
import pickle 
fp = os.path.join(data_path, 'data_inter', 'dem_array_data.pkl')

with open(fp, 'rb') as handle:
    dem_array_data = pickle.load(handle)

In [ ]:
grid_res = dem_array_data['grid_res']
x_min, x_max, y_min, y_max = dem_array_data['bounds']
xx, yy = dem_array_data['mesh_xy']
dem_array_zeroes_infill = dem_array_data['dem_array_zeroes_infill']
dem_array = dem_array_data['dem_array']

In [ ]:
mlab.figure(size=(800, 800))
mlab.surf(xx, yy, dem_array, warp_scale=20, colormap='terrain')
mlab.outline()
mlab.show()

In [ ]:
grid

In [ ]:
grid = pv.StructuredGrid(xx, yy, dem_array * 20)
grid.plot()

In [ ]:
# First a default plot with jet colormap
p = pv.Plotter()
# Add the data, use active scalar for coloring, and show the scalar bar
p.add_mesh(grid)
# Display the scene
p.show()

In [ ]:
grid.plot_curvature(clim=[-1, 1])